    This example uses US Stocks Fundamentals dataset available on Kaggle. 
    
    You can download it from https://www.kaggle.com/usfundamentals/us-stocks-fundamentals
    
    You need to install Cassandra driver to run this example.
    
    pip install cassandra-driver

In [1]:
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(['localhost'])

In [3]:
session = cluster.connect()

In [11]:
##creating a keyspace 
session.execute("CREATE KEYSPACE stocks \
                    WITH replication = {'class': 'SimpleStrategy', \
                                        'replication_factor': '1'}")

In [4]:
##telling session to work on keyspace
session.set_keyspace('stocks')

In [13]:
## create a table company to load data
session.execute("""
                CREATE TABLE company (
                        company_id text,
                        name_latest text,
                        name_previous text,
                        PRIMARY KEY (company_id)
                    )
                """)

In [14]:
session.execute("""
                CREATE TABLE indicator_by_company (
                        company_id text,
                        indicator_id text,
                        yr_2010 bigint,
                        yr_2011 bigint,
                        yr_2012 bigint,
                        yr_2013 bigint,
                        yr_2014 bigint,
                        yr_2015 bigint,
                        yr_2016 bigint,
                        PRIMARY KEY (company_id, indicator_id)
                    )
                """)

In [5]:
import json

In [6]:
## importing json data to be loaded
with open('companies.json') as f_in:
    companies = json.load(f_in)

In [17]:
type(companies)

list

In [18]:
companies[:5]

[{'company_id': '1000045',
  'name_latest': 'Nicholas Financial Inc',
  'names_previous': None},
 {'company_id': '1000180',
  'name_latest': 'Sandisk Corp',
  'names_previous': None},
 {'company_id': '1000209',
  'name_latest': 'Medallion Financial Corp',
  'names_previous': None},
 {'company_id': '1000228',
  'name_latest': 'Henry Schein Inc',
  'names_previous': None},
 {'company_id': '1000229',
  'name_latest': 'Core Laboratories N V',
  'names_previous': None}]

In [22]:
for company in companies:
    try:
        session.execute("""
            INSERT INTO company(company_id, name_latest, name_previous)
            VALUES(%s, %s, %s)
        """, (company['company_id'], company['name_latest'], company['names_previous']))
    except:
        pass

In [7]:
result_set = session.execute("SELECT * FROM company")

In [8]:
result_set.current_rows

[Row(company_id='1445229', name_latest='Fuer International Inc.', name_previous=None),
 Row(company_id='1376866', name_latest='8888 Acquisition Corp', name_previous=None),
 Row(company_id='76267', name_latest='Park Electrochemical Corp', name_previous=None),
 Row(company_id='1360334', name_latest='Alamo Energy Corp.', name_previous=None),
 Row(company_id='832101', name_latest='Idex Corp /DE/', name_previous=None),
 Row(company_id='1645113', name_latest='Novocure LTD', name_previous=None),
 Row(company_id='1043219', name_latest='Annaly Capital Management Inc', name_previous=None),
 Row(company_id='78003', name_latest='Pfizer Inc', name_previous=None),
 Row(company_id='1396033', name_latest='Lumber Liquidators Holdings, Inc.', name_previous=None),
 Row(company_id='1438945', name_latest='Strike Axe, Inc', name_previous=None),
 Row(company_id='828750', name_latest='GSI Commerce Inc', name_previous=None),
 Row(company_id='1140028', name_latest='Talon Therapeutics, Inc.', name_previous=None)

In [9]:
import csv

In [15]:
def safe_int(str_val: str): 
    try:
        if str_val == '':
            return(0)
        else:
            return(int(str_val))
    except:
        return(0)

In [19]:
with open('indicators_by_company.csv') as f_in:
    indicators = csv.DictReader(f_in)
    for indicator in indicators:
        session.execute(
         """
        INSERT INTO indicator_by_company (company_id, indicator_id, yr_2010, yr_2011, yr_2012, yr_2013, yr_2014, yr_2015, yr_2016)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (indicator['company_id'], indicator['indicator_id'], safe_int(indicator['2010']), safe_int(indicator['2011']), safe_int(indicator['2012']), safe_int(indicator['2013']), safe_int(indicator['2014']), safe_int(indicator['2015']), safe_int(indicator['2016'])))

PermissionError: [Errno 13] Permission denied

In [20]:
result_set = session.execute("SELECT * FROM indicator_by_company")
result_set.current_rows

[Row(company_id='101199', indicator_id='AccretionAmortizationOfDiscountsAndPremiumsInvestments', yr_2010=0, yr_2011=-11638000, yr_2012=-15183000, yr_2013=-15291000, yr_2014=-14434000, yr_2015=-13745000, yr_2016=0),
 Row(company_id='101199', indicator_id='AccruedIncomeTaxes', yr_2010=0, yr_2011=0, yr_2012=0, yr_2013=0, yr_2014=5012000, yr_2015=4917000, yr_2016=0),
 Row(company_id='101199', indicator_id='AccruedInvestmentIncomeReceivable', yr_2010=0, yr_2011=32219000, yr_2012=30375000, yr_2013=27923000, yr_2014=25989000, yr_2015=25136000, yr_2016=0),
 Row(company_id='101199', indicator_id='AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment', yr_2010=0, yr_2011=35248000, yr_2012=34982000, yr_2013=36972000, yr_2014=41492000, yr_2015=46590000, yr_2016=0),
 Row(company_id='101199', indicator_id='AccumulatedOtherComprehensiveIncomeLossNetOfTax', yr_2010=0, yr_2011=82586000, yr_2012=95188000, yr_2013=87150000, yr_2014=91173000, yr_2015=80437000, yr_2016=0),
 Row(company_i